In [43]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import math
%matplotlib inline

In [56]:
#读取数据集
import pandas as pd
xy = pd.read_csv('diabetes.csv',delimiter=',',dtype= np.float32)
xy_numpy = xy.to_numpy() #pandas 转维numpy 为了后面numpy转tensor
x = xy_numpy[:,0:-1] # x为 768*8
y = xy_numpy[:,-1].reshape(-1,1) #为了让其shape为 768*1 ，而不是768

[ 1.    85.    66.    29.     0.    26.6    0.351 31.     0.   ]
[ 1.    85.    66.    29.     0.    26.6    0.351 31.   ]


In [45]:
#numpy转Tensor
x_data = torch.Tensor(torch.from_numpy(x)) #注意此处的Tensor 若由numpy转换过来 numpy的dtype须为np.float32或其他float类型
y_data = torch.Tensor(torch.from_numpy(y))

In [46]:
print(x_data.data.shape)
print(y_data.data.shape)

torch.Size([768, 8])
torch.Size([768, 1])


In [47]:
#搭建多层神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        # 定义多层神经网络
        self.fc1 = torch.nn.Linear(8,6)
        self.fc2 = torch.nn.Linear(6,4)
        self.fc3 = torch.nn.Linear(4,1)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))            # 8->6  第一层：
        x = F.dropout(x,p=0.5)             #dropout 1 
        x = F.relu(self.fc2(x))            #-6->4   第二层
        x = F.dropout(x,p=0.5)             # dropout 2
        y_pred = torch.sigmoid(self.fc3(x))         # 4->1 ->sigmoid  第三层+sigmoid层
        # warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead."
        return y_pred

In [57]:
#自定义权重初始化函数
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear')!= -1:
        m.weight.data = torch.randn(m.weight.data.size()[0],m.weight.data.size()[1])
        m.bias.data = torch.randn(m.bias.data.size()[0])

In [58]:
#实例化类
model = Model()
model.apply(weight_init)

Model(
  (fc1): Linear(in_features=8, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=1, bias=True)
)

In [42]:
#定义损失函数及优化器
criterion  = torch.nn.BCELoss() #定义损失函数 binary corsstropy
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01) #学习率设置为0.01,学习率为超参数 ，可以自己设置
Loss = []
print(x.shape)

(759, 8)


In [50]:
#训练
for epoch in range(2000):
    y_pred = model(x_data)
    #计算误差
    loss = criterion(y_pred,y_data)
    #
    #prin(loss.item())
    Loss.append(loss.item())
    #每迭代1000次打印Lost并记录
    if epoch%100 == 0:
        print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, 2000, loss.item()))
    #梯度清零
    optimizer.zero_grad()
    #反向传播
    loss.backward()
    #更新梯度
    optimizer.step()

[1,  2000] loss: 2.857
[101,  2000] loss: 2.704
[201,  2000] loss: 2.650
[301,  2000] loss: 2.849
[401,  2000] loss: 2.887
[501,  2000] loss: 2.765
[601,  2000] loss: 3.028
[701,  2000] loss: 2.718
[801,  2000] loss: 2.763
[901,  2000] loss: 2.993
[1001,  2000] loss: 3.094
[1101,  2000] loss: 2.931
[1201,  2000] loss: 3.117
[1301,  2000] loss: 3.129
[1401,  2000] loss: 2.876
[1501,  2000] loss: 2.897
[1601,  2000] loss: 2.872
[1701,  2000] loss: 3.050
[1801,  2000] loss: 2.982
[1901,  2000] loss: 2.919


In [51]:
for i in range(len(y_pred)):
    if(y_pred[i]>0.5):
        y_pred[i] = 1.0
    else:
        y_pred[i] = 0.0
#print(y_pred)
type(y_pred)

torch.Tensor

In [52]:
#准确率：
(y_pred == y_data).sum().item()/len(y_data) # torch.Tensor.sum()函数

0.5104166666666666